## 前処理目的<a name='S4_e'> 
* 住民基本台帳と人口動態調査を用いた人口・死亡・出生における分析用データの作成

<br>

## 各種ステップ

### [STEP0 事前準備](#STEP0)
1. 必要なモジュールのimport
2. 市区町村マスタの読み込み
3. 政令指定都市マスタの読み込み

### [STEP1 住民基本台帳による人口データの前処理](#STEP1)
1. 指定フォルダにあるファイル情報の抽出（リスト、grob、ワイルドカード）
2. ファイルの読み込み（read_csv、type、assign、pd.concat）
3. 必要なカラムのみに制限
4. 文字列から数値への変換（dtype、連鎖代入、copy()、str.replace、astype）
5. 市区町村コードの追加（str[]）
6. 不要な行の除去①市区町村コードが欠損（dropna）
7. 不要な行の除去②都道府県集計を除去=末尾が"000"（!、条件抽出）
8. 不要な行の除去③政令指定都市を除去（リストを用いた条件抽出）
9. 最新市区町村情報の付与（pd.merge(how=***、on=***)）
10. 最新市区町村でのグループ集計（groupby().統計量）

### [STEP2 人口動態調査による人口データの前処理](#STEP2)
1. 指定フォルダにあるファイル情報の抽出
2. ファイルの一括読み込み
3. 文字列から数値への変換（余計な文字列データの除去（全角スペース\u3000））
4. 市区町村コードの作成
5. 集計を最小単位にする（str.isdecimal()）
6. 最新市区町村へのマージ
7. 最新市区町村での再集計

### [STEP3 人口に関する可視化向けファイルの作成](#STEP3)
1. 住民基本台帳と人口動態調査の前処理データの横結合
2. 不要な行を削除
3. STEP3-2のデータから政令指定都市のみを抽出
4. 政令指定都市で再集計
5. 政令指定都市以外の集計データとの縦結合
6. 出生率・死亡率の算出
7. カラム名の修正
8. データの並び替え政令指定都市以外の集計データとの縦結合
9. ファイルの出力

<br>

## [章末問題【解答例】](#EXE4)

##### この章末問題は，本文での処理を実施し作成されたデータフレームを利用する。<br><br><font color="red" weight="bold">※本ノートブックではデータフレームをファイルとして読み替えること.<br>　また, データフレームの変数名dfx_xxxの内容はファイル名dfx_xxx.csvに出力されており, <br>　そのファイルのパス文字列が変数dfx_xxxに格納されている.</font><br>

#### [1. データフレームdf3_outを用いて，集計年2018年における，出生率TOP20となる市区町村を求めなさい。](#EXE4_1)

##### なお，ファイル名を「2018年出生率TOP20[本日日付] [自分の名前].csv」，カラムを「出生率ランキング」を元のデータフレームに追加とする。<br>


#### [2. データフレームdf3_base2を用いて，都道府県集計のデータフレームを作成しなさい。](#EXE4_2)<br>

#### [3. データフレームdf2_base3から，都道府県集計のレコードを抽出しなさい。](#EXE4_3)<br>

#### [4. 問題(2) と問題(3) の結果を比較し，差異が2010年神奈川県に存在することを確認しなさい。](#EXE4_4)<br>

#### [5. 問題(4) にて，差分が2010年神奈川県にのみ存在した理由を述べなさい。](#EXE4_5)<br>


### STEP0 事前準備<a name='STEP0'>

In [1]:
#{コード4.1 事前準備}
#必要なライブラリの読み込み
import os 
from glob import glob
import datetime
import nysol.mcmd as nm
import codecs
import xlrd
#実行環境によって必要なxlrdライブラリの設定を行う
if hasattr(xlrd, 'xlsx'):
    xlrd.xlsx.ensure_elementtree_imported(False, None)
    xlrd.xlsx.Element_has_iter = True
    
os.makedirs('out_m',exist_ok=True)

* nysol_pythonの入出力ファイルは, 文字コード=UTF-8, 改行コード=LFに限定されている.
* よって, 事前にShift_JISのファイルをUTF-8（改行コードはLF）に変換する.

In [2]:
#テキストファイルの文字コードをShift_JISからUTF-8（改行コードはLF）に変換する関数
def sjisToUTF8(infile, outfile):
    with codecs.open(infile, 'r', 'shift_jis') as fpr:
        with codecs.open(outfile, 'w', 'utf-8') as fpw:
            for row in fpr:
                fpw.write(row.replace('\r',''))

In [3]:
#出力先ディレクトリ作成
out_dir='./out_m/step0/'
os.makedirs(out_dir, exist_ok=True)

#マスタファイルの文字コードをshift_jisからutf-8に変換
sjisToUTF8('./in/municipalitiy_mst.csv', out_dir+'municipalitiy_mst.csv')
sjisToUTF8('./in/seireicity_mst.csv', out_dir+'seireicity_mst.csv')

#人口動態のファイルの文字コードをshift_jisからutf-8に変換
for current_dir, subdir, files in os.walk('./in/jinkodoutai'):
    for infile in glob(os.path.join(current_dir, '*.csv')):
        outfile=infile.replace('/in/','/out_m/step0/')
        outdir=os.path.dirname(outfile)
        os.makedirs(outdir, exist_ok=True)
        sjisToUTF8(infile, outfile)

* また, nysol_pythonでは事前にマスタファイルをメモリ上に読み込んでおく必要がないため, ここではマスタファイルをnysol_pythonで正しく読み込めるか確認するのみとする.

In [4]:
# {コード4.2 市区町村マスタの読み込み}

#nm.p()メソッドは引数のCSVファイルの内容を表示する. head, tailパラメータで先頭, 末尾からの表示行数を指定できる.
#  末尾の行を表示することで行インデックスの値からファイルの行数を確認することができる.
#   ※行インデックスは0からスタートするため, 末尾の行インデックスに+1したものが行数となる.

j5_mst='./out_m/step0/municipalitiy_mst.csv'
display(nm.p(j5_mst, head=5, tail=1))

,変換前市区町村コード,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,政令指定都市フラグ
0,01100,01,北海道,01100,札幌市,1
1,01101,01,北海道,01101,札幌市中央区,
2,01102,01,北海道,01102,札幌市北区,
3,01103,01,北海道,01103,札幌市東区,
4,01104,01,北海道,01104,札幌市白石区,
...,...,...,...,...,...,...
424,47382,47,沖縄県,47382,与那国町,


In [5]:
# {コード4.3 政令指定都市マスタの読み込み}
seirei_mst='./out_m/step0/seireicity_mst.csv'
display(nm.p(seirei_mst, head=5, tail=1))

,変換前市区町村コード,政令指定都市コード,政令指定都市名
0,01101,01100,札幌市
1,01102,01100,札幌市
2,01103,01100,札幌市
3,01104,01100,札幌市
4,01105,01100,札幌市
...,...,...,...
37,14153,14150,相模原市


* EXCELファイルで提供されている住民基本台帳のデータをセル単位に展開したCSVファイルに変換する.

In [6]:
#EXCELファイルを読み込んでセル単位に展開したCSVファイルに変換する関数
# CSVファイルの項目は｛ブック名,シート名,行番号,列番号,列名,セル名,セル型,セル値｝
def excelToCellCSV(_book_name, _dst_name=None):
    with xlrd.open_workbook(_book_name) as _book: #Excelファイルオープン
        if _dst_name is None or _dst_name == '':
            #出力ファイル名が未設定の場合は入力ファイルと同じ場所に拡張子がcsvの同じ名前のファイルを出力
            _ext = os.path.splitext(os.path.basename(_book_name))[1]
            _dst_name = _book_name.replace(_ext, ".csv")
        with open(_dst_name, mode="w") as _f: #出力CSVファイルオープン
            #CSVヘッダー（項目名）書き込み
            _f.write("ブック名,シート名,行番号,列番号,列名,セル名,セル型,セル値\n")

            for _sheet_name in _book.sheet_names():
                _sheet = _book.sheet_by_name(_sheet_name)
                for _row in range(_sheet.nrows):
                    for _col in range(_sheet.ncols):
                        _cell = _sheet.cell(_row,_col)
                        if _cell.ctype == xlrd.XL_CELL_EMPTY: #0
                            _cell_value = ''
                        elif _cell.ctype == xlrd.XL_CELL_TEXT: #1
                            #先頭・末尾の空白・改行・タブ文字、全角空白を削除
                            #全角スペースの文字コード表現(\u3000)を文字に変換
                            #値内部に改行コードなどが含まれているケースのためにダブルクォーテーションで括る
                            #  →それをうけてダブルクォーテーション1つは2つ（""）に変換）
                            _cell_value = "\""+_cell.value.strip().strip('　').replace('\u3000','　').replace('"','""')+"\""
                        elif _cell.ctype == xlrd.XL_CELL_NUMBER: #2
                            _str_val = str(_cell.value)
                            _cell_value = _str_val.replace('.0','') if _str_val.endswith('.0') else _str_val
                        elif _cell.ctype == xlrd.XL_CELL_DATE: #3
                            _cell_value = xlrd.xldate.xldate_as_datetime(_cell.value,0).strftime('%Y%m%d%H%M%S').replace('000000','')
                        elif _cell.ctype == xlrd.XL_CELL_BOOLEAN: #4
                            _cell_value = True if _cell.value == 1 else False
                        elif _cell.ctype == xlrd.XL_CELL_ERROR: #5
                            _cell_value = '#N/A'
                        else:
                            _cell_value = ''
                        #セルが空もしくは空白文字のみの場合は出力しない
                        if _cell_value != '' and _cell_value != '""':
                            _cell_info = ["\""+_book_name.replace('\u3000',' ').replace('"','""')+"\"",
                                          "\""+_sheet_name.replace('\u3000',' ').replace('"','""')+"\"",
                                          _row+1,
                                          _col+1,
                                          xlrd.formula.colname(_col),
                                          xlrd.formula.cellname(_row, _col),
                                          _cell.ctype,
                                          _cell_value]                    
                            _f.write(",".join(map(str,_cell_info))+"\n")

In [7]:
#EXCELファイル変換後のCSVファイル出力先ディレクトリを作成
os.makedirs('./out_m/step0/juuminkihon/cell_csv', exist_ok=True)

for xlsfile in glob('./in/juuminkihon/*'):
    #出力ファイル名を生成
    #  出力先ディレクトリをinからout_mに、ファイルの拡張子をxlsからcsvに変換 
    #   （例） ./in/juuminkihon/0903ssjin.xls  → ./out_m/juuminkihon/0903ssjin.csv
    cellcsvfile = xlsfile.replace('./in/juuminkihon/', './out_m/step0/juuminkihon/cell_csv/').replace('.xls', '.csv')
    excelToCellCSV(xlsfile, cellcsvfile)

    csvfile = xlsfile.replace('./in/juuminkihon/', './out_m/step0/juuminkihon/').replace('.xls', '.csv')
    f=None
    f<<=nm.m2cross(i=cellcsvfile, k='ブック名,シート名,行番号', s='列番号%n', f='セル値')
    f<<=nm.msortf(f='ブック名,シート名,行番号%n')
    f<<=nm.mcut(r=True, f='ブック名,シート名,列名,セル名,セル型', o=csvfile)
    f.run(msg="on")


#END# kg2cross f=セル値 i=./out_m/step0/juuminkihon/cell_csv/0903ssjin.csv k=ブック名,シート名,行番号 s=列番号%n; IN=7890 OUT=379; 2021/05/18 23:42:34; 2021/05/18 23:42:34
#END# kgcut -r f=ブック名,シート名,列名,セル名,セル型 o=./out_m/step0/juuminkihon/0903ssjin.csv; IN=379 OUT=379; 2021/05/18 23:42:34; 2021/05/18 23:42:34
#END# kgsortf f=ブック名,シート名,行番号%n; IN=379 OUT=379; 2021/05/18 23:42:34; 2021/05/18 23:42:34
#END# kg2cross f=セル値 i=./out_m/step0/juuminkihon/cell_csv/1607nsjin.csv k=ブック名,シート名,行番号 s=列番号%n; IN=11719 OUT=381; 2021/05/18 23:42:34; 2021/05/18 23:42:34
#END# kgcut -r f=ブック名,シート名,列名,セル名,セル型 o=./out_m/step0/juuminkihon/1607nsjin.csv; IN=381 OUT=381; 2021/05/18 23:42:34; 2021/05/18 23:42:34
#END# kgsortf f=ブック名,シート名,行番号%n; IN=381 OUT=381; 2021/05/18 23:42:34; 2021/05/18 23:42:34
#END# kg2cross f=セル値 i=./out_m/step0/juuminkihon/cell_csv/1307nsjin.csv k=ブック名,シート名,行番号 s=列番号%n; IN=11719 OUT=381; 2021/05/18 23:42:35; 2021/05/18 23:42:35
#END# kgsortf f=ブック名,シート名,行番号%n; IN=381 OUT=381; 2021/05/18 23:42:35; 2021/05

### STEP1 住民基本台帳による人口データの前処理<a name='STEP1'>

In [8]:
fldname_s = ['団体コード','都道府県名','市区町村名','人口_男','人口_女','人口_計','世帯数計','転入者数_計','出生者数','住民票記載数_その他_計','住民票記載数計','転出者数_計','死亡者数','住民票消除数_その他_計','住民票消除数計','住民票増減数','住民票増加率','自然増加数','自然増加率','社会増加数','社会増加率']
fldname_n = ['団体コード','都道府県名','市区町村名','人口_男','人口_女','人口_計','世帯数_日本人住民','世帯数_複数国籍','世帯数計','転入者数_国内','転入者数_国外','転入者数_計','出生者数','住民票記載数_その他_帰化等','住民票記載数_その他_その他','住民票記載数_その他_計','住民票記載数計','転出者数_国内','転出者数_国外','転出者数_計','死亡者数','住民票消除数_その他_国籍喪失','住民票消除数_その他_その他','住民票消除数_その他_計','住民票消除数計','住民票増減数','住民票増加率','自然増加数','自然増加率','社会増加数','社会増加率']
str_cfname_s = ','.join([str(i+1)+':'+v   for i,v in enumerate(fldname_s)])
str_cfname_n = ','.join([str(i+1)+':'+v   for i,v in enumerate(fldname_n)])

os.makedirs('./out_m/step1/juuminkihon', exist_ok=True)
for infile in glob('./out_m/step0/juuminkihon/*.csv'):
    if 'ssjin.csv' in infile:
        str_cfname=str_cfname_s
    else:
        str_cfname=str_cfname_n
    outfile=infile.replace('/step0/', '/step1/')
    f=None
    f<<=nm.msel(i=infile, c='${行番号}>4')
    f<<=nm.mfldname(f=str_cfname)
    f.writecsv(o=outfile).run(msg="on")
    display(outfile)
    display(nm.p(outfile, head=3, tail=1))

    
file1_base1='./out_m/step1/df1_base1.csv'
    
names=','.join(set(fldname_s) |  {'行番号'})
f=None
f<<=nm.mcat(i='./out_m/step1/juuminkihon/*.csv', f=names, a='-1:fileName')
f<<=nm.mcal(c='"20"+left($s{fileName},2)',a='集計年', o=file1_base1)
f.run(msg="on")

display(file1_base1)
display(nm.p(file1_base1, head=5, tail=1))

#END# kgsel c=${行番号}>4 i=./out_m/step0/juuminkihon/1707nsjin.csv; IN=381 OUT=377; 2021/05/18 23:42:39; 2021/05/18 23:42:39
#END# kgfldname f=1:団体コード,2:都道府県名,3:市区町村名,4:人口_男,5:人口_女,6:人口_計,7:世帯数_日本人住民,8:世帯数_複数国籍,9:世帯数計,10:転入者数_国内,11:転入者数_国外,12:転入者数_計,13:出生者数,14:住民票記載数_その他_帰化等,15:住民票記載数_その他_その他,16:住民票記載数_その他_計,17:住民票記載数計,18:転出者数_国内,19:転出者数_国外,20:転出者数_計,21:死亡者数,22:住民票消除数_その他_国籍喪失,23:住民票消除数_その他_その他,24:住民票消除数_その他_計,25:住民票消除数計,26:住民票増減数,27:住民票増加率,28:自然増加数,29:自然増加率,30:社会増加数,31:社会増加率; IN=377 OUT=377; 2021/05/18 23:42:39; 2021/05/18 23:42:39
#END# kgload o=./out_m/step1/juuminkihon/1707nsjin.csv; IN=378 OUT=377; 2021/05/18 23:42:39; 2021/05/18 23:42:39


'./out_m/step1/juuminkihon/1707nsjin.csv'

,行番号,団体コード,都道府県名,市区町村名,人口_男,人口_女,人口_計,世帯数_日本人住民,世帯数_複数国籍,世帯数計,転入者数_国内,転入者数_国外,転入者数_計,出生者数,住民票記載数_その他_帰化等,住民票記載数_その他_その他,住民票記載数_その他_計,住民票記載数計,転出者数_国内,転出者数_国外,転出者数_計,死亡者数,住民票消除数_その他_国籍喪失,住民票消除数_その他_その他,住民票消除数_その他_計,住民票消除数計,住民票増減数,住民票増加率,自然増加数,自然増加率,社会増加数,社会増加率
0,5,,合計,,61290165,64293493,125583658,55767195,454373,56221568,4877759,170354,5048113,981202,10074,66905,76979,6106294,4890205,174553,5064758,1309515,221,39884,40105,6414378,-308084,-0.24472137338444327,-328313,-0.260789941249681,20229,0.016068567865237735
1,6,010006,北海道,,2525160,2817458,5342618,2737588,5014,2742602,240408,3916,244324,35322,61,1972,2033,281679,248286,4063,252349,62068,16,839,855,315272,-33593,-0.6248452674197497,-26746,-0.4974879148158433,-6847,-0.12735735260390635
2,7,011002,北海道,札幌市,905718,1030455,1936173,1028580,2443,1031023,118104,2024,120128,14107,27,568,595,134830,109166,2114,111280,18567,15,313,328,130175,4655,0.24100215478188658,-4460,-0.23090646838393428,9115,0.4719086231658209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,381,473821,沖縄県,八重山郡与那国町,924,773,1697,913,5,918,376,2,378,18,0,0,0,396,165,0,165,19,0,0,0,184,212,14.276094276094275,-1,-0.06734006734006734,213,14.343434343434344


#END# kgsel c=${行番号}>4 i=./out_m/step0/juuminkihon/1103ssjin.csv; IN=381 OUT=377; 2021/05/18 23:42:39; 2021/05/18 23:42:39
#END# kgload o=./out_m/step1/juuminkihon/1103ssjin.csv; IN=378 OUT=377; 2021/05/18 23:42:39; 2021/05/18 23:42:39
#END# kgfldname f=1:団体コード,2:都道府県名,3:市区町村名,4:人口_男,5:人口_女,6:人口_計,7:世帯数計,8:転入者数_計,9:出生者数,10:住民票記載数_その他_計,11:住民票記載数計,12:転出者数_計,13:死亡者数,14:住民票消除数_その他_計,15:住民票消除数計,16:住民票増減数,17:住民票増加率,18:自然増加数,19:自然増加率,20:社会増加数,21:社会増加率; IN=377 OUT=377; 2021/05/18 23:42:39; 2021/05/18 23:42:39


'./out_m/step1/juuminkihon/1103ssjin.csv'

...

'./out_m/step1/juuminkihon/1407nsjin.csv'

,行番号,団体コード,都道府県名,市区町村名,人口_男,人口_女,人口_計,世帯数_日本人住民,世帯数_複数国籍,世帯数計,転入者数_国内,転入者数_国外,転入者数_計,出生者数,住民票記載数_その他_帰化等,住民票記載数_その他_その他,住民票記載数_その他_計,住民票記載数計,転出者数_国内,転出者数_国外,転出者数_計,死亡者数,住民票消除数_その他_国籍喪失,住民票消除数_その他_その他,住民票消除数_その他_計,住民票消除数計,住民票増減数,住民票増加率,自然増加数,自然増加率,社会増加数,社会増加率
0,5,,合計,,61727463,64707171,126434634,54514453,437553,54952006,5005058,159081,5164139,1030388,9095,73177,82272,6276799,5027606,173642,5201248,1267838,180,51547,51727,6520813,-244014,-0.19262441133726024,-237450,-0.187442796200351,-6564,-0.0051816151369092605
1,6,010006,北海道,,2575347,2865732,5441079,2709054,4671,2713725,251475,3386,254861,38271,53,2164,2217,295349,260842,3715,264557,59482,5,944,949,324988,-29639,-0.5417753208993774,-21211,-0.38771876013349615,-8428,-0.1540565607658812
2,7,011002,北海道,札幌市,901232,1019838,1921070,998250,2258,1000508,123079,1642,124721,14602,26,615,641,139964,113318,1833,115151,16784,5,248,253,132188,7776,0.40641950479121347,-2182,-0.1140441563084398,9958,0.5204636610996533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,381,473821,沖縄県,八重山郡与那国町,791,752,1543,785,6,791,120,0,120,15,0,0,0,135,132,0,132,16,0,0,0,148,-13,-0.8354755784061696,-1,-0.06426735218508997,-12,-0.7712082262210797


#END# kgcat a=-1:fileName f=社会増加率,住民票増減数,人口_女,市区町村名,人口_計,世帯数計,住民票記載数_その他_計,転出者数_計,自然増加数,住民票消除数_その他_計,団体コード,行番号,転入者数_計,住民票増加率,都道府県名,住民票消除数計,社会増加数,出生者数,死亡者数,人口_男,自然増加率,住民票記載数計 i=./out_m/step1/juuminkihon/*.csv; IN=3765 OUT=3765; 2021/05/18 23:42:39; 2021/05/18 23:42:39
#END# kgcal a=集計年 c="20"+left($s{fileName},2) o=./out_m/step1/df1_base1.csv; IN=3765 OUT=3765; 2021/05/18 23:42:39; 2021/05/18 23:42:39


'./out_m/step1/df1_base1.csv'

,社会増加率,住民票増減数,人口_女,市区町村名,人口_計,世帯数計,住民票記載数_その他_計,転出者数_計,自然増加数,住民票消除数_その他_計,団体コード,行番号,転入者数_計,住民票増加率,都道府県名,住民票消除数計,社会増加数,出生者数,死亡者数,人口_男,自然増加率,住民票記載数計,fileName,集計年
0,0.016068567865237735,-308084,64293493,,125583658,56221568,76979,5064758,-328313,40105,,5,5048113,-0.24472137338444327,合計,6414378,20229,981202,1309515,61290165,-0.260789941249681,6106294,1707nsjin.csv,2017
1,-0.12735735260390635,-33593,2817458,,5342618,2742602,2033,252349,-26746,855,010006,6,244324,-0.6248452674197497,北海道,315272,-6847,35322,62068,2525160,-0.4974879148158433,281679,1707nsjin.csv,2017
2,0.4719086231658209,4655,1030455,札幌市,1936173,1031023,595,111280,-4460,328,011002,7,120128,0.24100215478188658,北海道,130175,9115,14107,18567,905718,-0.23090646838393428,134830,1707nsjin.csv,2017
3,0.8243781007310275,1628,125850,札幌市中央区,229800,136754,97,18852,-253,30,011011,8,20666,0.7134968357204214,北海道,20884,1881,1749,2002,103950,-0.11088126501060604,22512,1707nsjin.csv,2017
4,0.6664435694603659,1156,148289,札幌市北区,282050,147421,95,14048,-716,27,011029,9,15852,0.4115431443889866,北海道,16641,1872,1850,2566,133761,-0.2549004250713792,17797,1707nsjin.csv,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3764,-0.7712082262210797,-13,752,八重山郡与那国町,1543,791,0,132,-1,0,473821,381,120,-0.8354755784061696,沖縄県,148,-12,15,16,791,-0.06426735218508997,135,1407nsjin.csv,2014


In [9]:
#{コード4.6 必要なカラムのみに制限}
file1_base2='./out_m/step1/df1_base2.csv'

f=None
f<<=nm.mcut(i=file1_base1, f='集計年,団体コード,人口_計', o=file1_base2)
f.run(msg="on")

display(file1_base2)
display(nm.p(file1_base2, head=5, tail=1))

#END# kgcut f=集計年,団体コード,人口_計 i=./out_m/step1/df1_base1.csv o=./out_m/step1/df1_base2.csv; IN=3765 OUT=3765; 2021/05/18 23:42:40; 2021/05/18 23:42:40


'./out_m/step1/df1_base2.csv'

,集計年,団体コード,人口_計
0,2017,,125583658
1,2017,010006,5342618
2,2017,011002,1936173
3,2017,011011,229800
4,2017,011029,282050
...,...,...,...
3764,2014,473821,1543


In [10]:
# {コード4.7 市区町村コードの追加}
file1_base3='./out_m/step1/df1_base3.csv'

f=None
f<<=nm.mcal(i=file1_base2, c='left($s{団体コード},5)', a='市区町村コード', o=file1_base3)
f.run(msg="on")

display(file1_base3)
display(nm.p(file1_base3, head=3, tail=1))

#END# kgcal a=市区町村コード c=left($s{団体コード},5) i=./out_m/step1/df1_base2.csv o=./out_m/step1/df1_base3.csv; IN=3765 OUT=3765; 2021/05/18 23:42:40; 2021/05/18 23:42:40


'./out_m/step1/df1_base3.csv'

,集計年,団体コード,人口_計,市区町村コード
0,2017,,125583658,
1,2017,010006,5342618,01000
2,2017,011002,1936173,01100
...,...,...,...,...
3764,2014,473821,1543,47382


In [11]:
# {コード4.8 NaN のレコードの除去1}
file1_base4='./out_m/step1/df1_base4.csv'

f=None
f<<=nm.mdelnull(i=file1_base3, f='市区町村コード', o=file1_base4)
f.run(msg="on")

display(file1_base4)
display(nm.p(file1_base4, head=3, tail=1))

#END# kgdelnull f=市区町村コード i=./out_m/step1/df1_base3.csv o=./out_m/step1/df1_base4.csv; IN=3765 OUT=3749; 2021/05/18 23:42:40; 2021/05/18 23:42:40


'./out_m/step1/df1_base4.csv'

,集計年,団体コード,人口_計,市区町村コード
0,2017,010006,5342618,01000
1,2017,011002,1936173,01100
2,2017,011011,229800,01101
...,...,...,...,...
3748,2014,473821,1543,47382


In [12]:
# {コード4.9 末尾が000 のレコードの除去2}
file1_base5='./out_m/step1/df1_base5.csv'

f=None
f<<=nm.msel(i=file1_base4, c='mid($s{団体コード},2,3)!="000"', o=file1_base5)
f.run(msg="on")

display(file1_base5)
display(nm.p(file1_base5, head=3, tail=1))

#END# kgsel c=mid($s{団体コード},2,3)!="000" i=./out_m/step1/df1_base4.csv o=./out_m/step1/df1_base5.csv; IN=3749 OUT=3709; 2021/05/18 23:42:40; 2021/05/18 23:42:40


'./out_m/step1/df1_base5.csv'

,集計年,団体コード,人口_計,市区町村コード
0,2017,011002,1936173,01100
1,2017,011011,229800,01101
2,2017,011029,282050,01102
...,...,...,...,...
3708,2014,473821,1543,47382


In [13]:
# {コード4.10 政令指定都市の除去}
file1_base6='./out_m/step1/df1_base6.csv'

f_seirei=None
f_seirei<<=nm.mselstr(i=j5_mst, f='政令指定都市フラグ', v='1')
display('政令指定都市')
display(f_seirei)

f=None
f<<=nm.mcommon(i=file1_base5, m=f_seirei, r=True, k='市区町村コード', K='最新市区町村コード', o=file1_base6)
f.run(msg="on")

display(file1_base6)
display(nm.p(file1_base6, head=3, tail=1))

'政令指定都市'

0,01100,01,北海道,01100,札幌市,1
1,14100,14,神奈川県,14100,横浜市,1
2,14130,14,神奈川県,14130,川崎市,1
3,14150,14,神奈川県,14150,相模原市,1


#END# kgselstr f=政令指定都市フラグ i=./out_m/step0/municipalitiy_mst.csv v=1; IN=425 OUT=4; 2021/05/18 23:42:40; 2021/05/18 23:42:40
#END# kgcommon -r K=最新市区町村コード i=./out_m/step1/df1_base5.csv k=市区町村コード o=./out_m/step1/df1_base6.csv; IN=3709 OUT=3671; 2021/05/18 23:42:40; 2021/05/18 23:42:40


'./out_m/step1/df1_base6.csv'

,集計年,団体コード,人口_計,市区町村コード
0,2015,011011,224928,01101
1,2014,011011,222392,01101
2,2016,011011,228172,01101
...,...,...,...,...
3670,2014,473821,1543,47382


In [14]:
#{コード4.11 最新市区町村情報の付与}
file1_base7='./out_m/step1/df1_base7.csv'

f=None
f<<=nm.mjoin(i=file1_base6, m=j5_mst, k='市区町村コード', K='変換前市区町村コード', o=file1_base7)
f.run(msg="on")

display(file1_base7)
display(nm.p(file1_base7, head=10, tail=10))

#END# kgjoin K=変換前市区町村コード i=./out_m/step1/df1_base6.csv k=市区町村コード m=./out_m/step0/municipalitiy_mst.csv o=./out_m/step1/df1_base7.csv; IN=3671 OUT=3487; 2021/05/18 23:42:40; 2021/05/18 23:42:40


'./out_m/step1/df1_base7.csv'

,集計年,団体コード,人口_計,市区町村コード,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,政令指定都市フラグ
0,2015,011011,224928,01101,01,北海道,01101,札幌市中央区,
1,2014,011011,222392,01101,01,北海道,01101,札幌市中央区,
2,2016,011011,228172,01101,01,北海道,01101,札幌市中央区,
3,2011,011011,212666,01101,01,北海道,01101,札幌市中央区,
4,2012,011011,216307,01101,01,北海道,01101,札幌市中央区,
5,2010,011011,209934,01101,01,北海道,01101,札幌市中央区,
6,2018,011011,231394,01101,01,北海道,01101,札幌市中央区,
7,2017,011011,229800,01101,01,北海道,01101,札幌市中央区,
8,2013,011011,219066,01101,01,北海道,01101,札幌市中央区,
9,2009,011011,206252,01101,01,北海道,01101,札幌市中央区,


In [15]:
# {コード4.12 merge 実施時の確認例}
file1_che1='./out_m/step1/df1_che1.csv'
file1_che2='./out_m/step1/df1_che2.csv'
file1_che3='./out_m/step1/df1_che3.csv'

f=None
f<<=nm.mjoin(i=file1_base6, m=j5_mst, k='市区町村コード', K='変換前市区町村コード', n=True, o=file1_che1)
f.run(msg="on")
display(file1_che1)
display(nm.p(file1_che1, head=3, tail=1))

f=None
f<<=nm.mdelnull(i=file1_che1, r=True, f='最新市区町村コード', o=file1_che2)
f.run(msg="on")
display(file1_che2)
display(nm.p(file1_che2, head=3, tail=1))

f=None
f<<=nm.msel(i=file1_che2, c='mid($s{市区町村コード},3,1)!="3" && mid($s{市区町村コード},4,1)!="0"', o=file1_che3)
f.run(msg="on")
display(file1_che3)
display(nm.p(file1_che3, head=3, tail=1))

#END# kgjoin -n K=変換前市区町村コード i=./out_m/step1/df1_base6.csv k=市区町村コード m=./out_m/step0/municipalitiy_mst.csv o=./out_m/step1/df1_che1.csv; IN=3671 OUT=3671; 2021/05/18 23:42:40; 2021/05/18 23:42:40


'./out_m/step1/df1_che1.csv'

,集計年,団体コード,人口_計,市区町村コード,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,政令指定都市フラグ
0,2015,011011,224928,01101,01,北海道,01101,札幌市中央区,
1,2014,011011,222392,01101,01,北海道,01101,札幌市中央区,
2,2016,011011,228172,01101,01,北海道,01101,札幌市中央区,
...,...,...,...,...,...,...,...,...,...
3670,2014,473821,1543,47382,47,沖縄県,47382,与那国町,


#END# kgdelnull -r f=最新市区町村コード i=./out_m/step1/df1_che1.csv o=./out_m/step1/df1_che2.csv; IN=3671 OUT=184; 2021/05/18 23:42:40; 2021/05/18 23:42:40


'./out_m/step1/df1_che2.csv'

,集計年,団体コード,人口_計,市区町村コード,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,政令指定都市フラグ
0,2013,016951,0,01695,,,,,
1,2015,016951,0,01695,,,,,
2,2014,016951,0,01695,,,,,
...,...,...,...,...,...,...,...,...,...
183,2016,473804,5687,47380,,,,,


#END# kgsel c=mid($s{市区町村コード},3,1)!="3" && mid($s{市区町村コード},4,1)!="0" i=./out_m/step1/df1_che2.csv o=./out_m/step1/df1_che3.csv; IN=184 OUT=45; 2021/05/18 23:42:41; 2021/05/18 23:42:41


'./out_m/step1/df1_che3.csv'

,集計年,団体コード,人口_計,市区町村コード,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,政令指定都市フラグ
0,2013,016951,0,01695,,,,,
1,2015,016951,0,01695,,,,,
2,2014,016951,0,01695,,,,,
...,...,...,...,...,...,...,...,...,...
44,2010,016993,0,01699,,,,,


In [16]:
# {コード4.13 最新市区町村でのグループ集計}
file1_base8='./out_m/step1/df1_base8.csv'

f=None
f<<=nm.msum(i=file1_base7, k='集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名', f='人口_計')
f<<=nm.mcut(f='集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計', o=file1_base8)
f.run(msg="on")

display(file1_base8)
display(nm.p(file1_base8, head=10, tail=10))

#END# kgcut f=集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計 o=./out_m/step1/df1_base8.csv; IN=3486 OUT=3486; 2021/05/18 23:42:41; 2021/05/18 23:42:41
#END# kgsum f=人口_計 i=./out_m/step1/df1_base7.csv k=集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名; IN=3487 OUT=3486; 2021/05/18 23:42:41; 2021/05/18 23:42:41


'./out_m/step1/df1_base8.csv'

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計
0,2009,01,北海道,01101,札幌市中央区,206252
1,2009,01,北海道,01102,札幌市北区,273577
2,2009,01,北海道,01103,札幌市東区,252688
3,2009,01,北海道,01104,札幌市白石区,203579
4,2009,01,北海道,01105,札幌市豊平区,208476
5,2009,01,北海道,01106,札幌市南区,147397
6,2009,01,北海道,01107,札幌市西区,209883
7,2009,01,北海道,01108,札幌市厚別区,129604
8,2009,01,北海道,01109,札幌市手稲区,138794
9,2009,01,北海道,01110,札幌市清田区,114689


### STEP2 人口動態調査による人口データの前処理<a name='STEP2'>

In [17]:
#{コード4.14 指定フォルダにあるファイル情報の抽出}
current_dir = './out_m/step0/jinkodoutai'

all_csv_files2 = [file for current_dir, subdir, files in os.walk(current_dir)
                     for file in glob(os.path.join(current_dir, '*.csv'))]

all_csv_files2

['./out_m/step0/jinkodoutai/2013/a020047.csv',
 './out_m/step0/jinkodoutai/2013/a020013.csv',
 './out_m/step0/jinkodoutai/2013/a020001.csv',
 './out_m/step0/jinkodoutai/2013/a020014.csv',
 './out_m/step0/jinkodoutai/2014/a020047.csv',
 './out_m/step0/jinkodoutai/2014/a020013.csv',
 './out_m/step0/jinkodoutai/2014/a020001.csv',
 './out_m/step0/jinkodoutai/2014/a020014.csv',
 './out_m/step0/jinkodoutai/2015/a020047.csv',
 './out_m/step0/jinkodoutai/2015/a020013.csv',
 './out_m/step0/jinkodoutai/2015/a020001.csv',
 './out_m/step0/jinkodoutai/2015/a020014.csv',
 './out_m/step0/jinkodoutai/2012/a020047.csv',
 './out_m/step0/jinkodoutai/2012/a020013.csv',
 './out_m/step0/jinkodoutai/2012/a020001.csv',
 './out_m/step0/jinkodoutai/2012/a020014.csv',
 './out_m/step0/jinkodoutai/2009/a020047.csv',
 './out_m/step0/jinkodoutai/2009/a020013.csv',
 './out_m/step0/jinkodoutai/2009/a020001.csv',
 './out_m/step0/jinkodoutai/2009/a020014.csv',
 './out_m/step0/jinkodoutai/2017/a020047.csv',
 './out_m/ste

In [18]:
# {コード4.15 ファイルの一括読み込み}

# CSVファイルに整形
names=('地域情報',
       '出生数',
       '出生数_2500g未満',
       '死亡数',
       '死亡数_乳児',
       '死亡数_新生児',
       '死産数_自然死産',
       '死産数_人工死産',
       '周産期死亡_総数',
       '周産期死亡_22週以後の死産数',
       '周産期死亡_早期新生児死亡数',
       '婚姻件数',
       '離婚件数'
      )
for iFile in all_csv_files2:
    oFile=iFile.replace('/step0/', '/step2/')
    os.makedirs(os.path.dirname(oFile), exist_ok=True)
    with open(iFile) as f:
        skipRows=5
        if iFile.split('/')[-2]=='2010' and iFile.split('/')[-1]=='a020014.csv':
            skipRows=7
        d=f.read().split('\n')[skipRows:]
    with open(oFile,'w') as f:
        d=[','.join(names)]+d
        f.write('\n'.join(d))
        
# ファイルをすべて結合して集計年を項目追加
file2_base1='./out_m/step2/df2_base1.csv'

str_files=','.join([file.replace('/step0/', '/step2/') for file in all_csv_files2])

f=None
f<<=nm.mcat(i=str_files, a='-2:集計年', o=file2_base1)
f.run(msg="on")

display(file2_base1)
display(nm.p(file2_base1, head=5, tail=1))

#END# kgcat a=-2:集計年 i=./out_m/step2/jinkodoutai/2013/a020047.csv,./out_m/step2/jinkodoutai/2013/a020013.csv,./out_m/step2/jinkodoutai/2013/a020001.csv,./out_m/step2/jinkodoutai/2013/a020014.csv,./out_m/step2/jinkodoutai/2014/a020047.csv,./out_m/step2/jinkodoutai/2014/a020013.csv,./out_m/step2/jinkodoutai/2014/a020001.csv,./out_m/step2/jinkodoutai/2014/a020014.csv,./out_m/step2/jinkodoutai/2015/a020047.csv,./out_m/step2/jinkodoutai/2015/a020013.csv,./out_m/step2/jinkodoutai/2015/a020001.csv,./out_m/step2/jinkodoutai/2015/a020014.csv,./out_m/step2/jinkodoutai/2012/a020047.csv,./out_m/step2/jinkodoutai/2012/a020013.csv,./out_m/step2/jinkodoutai/2012/a020001.csv,./out_m/step2/jinkodoutai/2012/a020014.csv,./out_m/step2/jinkodoutai/2009/a020047.csv,./out_m/step2/jinkodoutai/2009/a020013.csv,./out_m/step2/jinkodoutai/2009/a020001.csv,./out_m/step2/jinkodoutai/2009/a020014.csv,./out_m/step2/jinkodoutai/2017/a020047.csv,./out_m/step2/jinkodoutai/2017/a020013.csv,./out_m/step2/jinkodoutai... tr

'./out_m/step2/df2_base1.csv'

,地域情報,出生数,出生数_2500g未満,死亡数,死亡数_乳児,死亡数_新生児,死産数_自然死産,死産数_人工死産,周産期死亡_総数,周産期死亡_22週以後の死産数,周産期死亡_早期新生児死亡数,婚姻件数,離婚件数,集計年
0,４７沖 縄,17209,1957,10956,30,23,219,224,77,66,11,8803,3651,2013
1,市 部,13439,1508,8266,23,17,171,180,59,50,9,6812,2887,2013
2,郡 部,3770,449,2690,7,6,48,44,18,16,2,1991,764,2013
3,4731那覇市,2652,273,1835,4,2,37,43,11,9,2,...,...,2013
4,4751北部,1130,132,1069,1,1,7,14,3,2,1,...,...,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4506,14402清川村,13,-,26,-,-,-,-,-,-,-,7,4,2016


In [19]:
# {コード4.16 文字列から数値への変換}
#スキル：余計な文字列データの除去（全角スペース\u3000, -, ...）
#数値への変換はnysol_pythonでは不要
file2_base2='./out_m/step2/df2_base2.csv'

f=None
f<<=nm.msed(i=file2_base1, f='*', c='[-|　|.]', v='', g=True, W=True)
f<<=nm.mnullto(f='*', v='0', o=file2_base2)
f.run(msg="on")

display(file2_base2)
display(nm.p(file2_base2, head=5, tail=1))

#END# kgsed -W -g c=[-|　|.] f=* i=./out_m/step2/df2_base1.csv v=; IN=4507 OUT=4507; 2021/05/18 23:42:41; 2021/05/18 23:42:41
#END# kgnullto f=* o=./out_m/step2/df2_base2.csv v=0; IN=4507 OUT=4507; 2021/05/18 23:42:41; 2021/05/18 23:42:41


'./out_m/step2/df2_base2.csv'

,地域情報,出生数,出生数_2500g未満,死亡数,死亡数_乳児,死亡数_新生児,死産数_自然死産,死産数_人工死産,周産期死亡_総数,周産期死亡_22週以後の死産数,周産期死亡_早期新生児死亡数,婚姻件数,離婚件数,集計年
0,４７沖縄,17209,1957,10956,30,23,219,224,77,66,11,8803,3651,2013
1,市部,13439,1508,8266,23,17,171,180,59,50,9,6812,2887,2013
2,郡部,3770,449,2690,7,6,48,44,18,16,2,1991,764,2013
3,4731那覇市,2652,273,1835,4,2,37,43,11,9,2,0,0,2013
4,4751北部,1130,132,1069,1,1,7,14,3,2,1,0,0,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4506,14402清川村,13,0,26,0,0,0,0,0,0,0,7,4,2016


In [20]:
# {コード4.17 市区町村コードの作成}
file2_base3='./out_m/step2/df2_base3.csv'

f=None
f<<=nm.mcal(i=file2_base2, c='leftw($s{地域情報},5)', a='市区町村コード', o=file2_base3)
f.run(msg="on")

display(file2_base3)
display(nm.p(file2_base3, head=5, tail=1))

#END# kgcal a=市区町村コード c=leftw($s{地域情報},5) i=./out_m/step2/df2_base2.csv o=./out_m/step2/df2_base3.csv; IN=4507 OUT=4507; 2021/05/18 23:42:41; 2021/05/18 23:42:41


'./out_m/step2/df2_base3.csv'

,地域情報,出生数,出生数_2500g未満,死亡数,死亡数_乳児,死亡数_新生児,死産数_自然死産,死産数_人工死産,周産期死亡_総数,周産期死亡_22週以後の死産数,周産期死亡_早期新生児死亡数,婚姻件数,離婚件数,集計年,市区町村コード
0,４７沖縄,17209,1957,10956,30,23,219,224,77,66,11,8803,3651,2013,４７沖縄
1,市部,13439,1508,8266,23,17,171,180,59,50,9,6812,2887,2013,市部
2,郡部,3770,449,2690,7,6,48,44,18,16,2,1991,764,2013,郡部
3,4731那覇市,2652,273,1835,4,2,37,43,11,9,2,0,0,2013,4731那
4,4751北部,1130,132,1069,1,1,7,14,3,2,1,0,0,2013,4751北
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4506,14402清川村,13,0,26,0,0,0,0,0,0,0,7,4,2016,14402


In [21]:
# {コード4.18 レコードを市区町村のみにする}
file2_base4='./out_m/step2/df2_base4.csv'

f=None
f<<=nm.msel(i=file2_base3, c='regexm($s{市区町村コード},"^[0-9]*$")', o=file2_base4)
f.run(msg="on")

display(file2_base4)
display(nm.p(file2_base4, head=5, tail=1))

#END# kgsel c=regexm($s{市区町村コード},"^[0-9]*$") i=./out_m/step2/df2_base3.csv o=./out_m/step2/df2_base4.csv; IN=4507 OUT=3492; 2021/05/18 23:42:41; 2021/05/18 23:42:41


'./out_m/step2/df2_base4.csv'

,地域情報,出生数,出生数_2500g未満,死亡数,死亡数_乳児,死亡数_新生児,死産数_自然死産,死産数_人工死産,周産期死亡_総数,周産期死亡_22週以後の死産数,周産期死亡_早期新生児死亡数,婚姻件数,離婚件数,集計年,市区町村コード
0,47201那覇市,3495,350,2492,4,2,49,55,18,16,2,1902,789,2013,47201
1,47205宜野湾市,1282,153,547,5,3,15,13,7,6,1,683,276,2013,47205
2,47207石垣市,666,63,402,1,1,9,10,2,1,1,334,173,2013,47207
3,47208浦添市,1445,153,642,2,2,15,21,9,8,1,772,281,2013,47208
4,47209名護市,763,87,497,1,1,6,12,2,1,1,373,146,2013,47209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3491,14402清川村,13,0,26,0,0,0,0,0,0,0,7,4,2016,14402


In [22]:
#{コード4.19 最新市区町村へのマージ}
file2_base5='./out_m/step2/df2_base5.csv'

f=None
f<<=nm.mjoin(i=file2_base4, m=j5_mst, k='市区町村コード', K='変換前市区町村コード', o=file2_base5)
f.run(msg="on")

display(file2_base5)
display(nm.p(file2_base5, head=5, tail=1))

#END# kgjoin K=変換前市区町村コード i=./out_m/step2/df2_base4.csv k=市区町村コード m=./out_m/step0/municipalitiy_mst.csv o=./out_m/step2/df2_base5.csv; IN=3492 OUT=3492; 2021/05/18 23:42:42; 2021/05/18 23:42:42


'./out_m/step2/df2_base5.csv'

,地域情報,出生数,出生数_2500g未満,死亡数,死亡数_乳児,死亡数_新生児,死産数_自然死産,死産数_人工死産,周産期死亡_総数,周産期死亡_22週以後の死産数,周産期死亡_早期新生児死亡数,婚姻件数,離婚件数,集計年,市区町村コード,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,政令指定都市フラグ
0,01101札幌市中央区,1750,148,1844,3,0,25,49,7,7,0,1716,563,2012,01101,01,北海道,01101,札幌市中央区,
1,01101中央区,1751,177,1679,2,1,17,40,6,5,1,1779,588,2010,01101,01,北海道,01101,札幌市中央区,
2,01101札幌市中央区,1706,178,1710,5,5,20,48,12,7,5,1672,612,2011,01101,01,北海道,01101,札幌市中央区,
3,01101中央区,1580,141,1571,2,0,14,38,3,3,0,1596,576,2009,01101,01,北海道,01101,札幌市中央区,
4,01101札幌市中央区,1874,170,1894,6,2,23,56,10,8,2,1672,658,2015,01101,01,北海道,01101,札幌市中央区,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3491,47382与那国町,12,1,15,0,0,1,2,0,0,0,13,3,2011,47382,47,沖縄県,47382,与那国町,


In [23]:
#{コード4.20 最新市区町村での再集計}
file2_base6='./out_m/step2/df2_base6.csv'

f=None
f<<=nm.msum(i=file2_base5, k='集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名', f='出生数,死亡数')
f<<=nm.mcut(f='集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,出生数,死亡数',  o=file2_base6)
f.run(msg="on")

display(file2_base6)
display(nm.p(file2_base6, head=5, tail=1))

#END# kgcut f=集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,出生数,死亡数 o=./out_m/step2/df2_base6.csv; IN=3489 OUT=3489; 2021/05/18 23:42:42; 2021/05/18 23:42:42
#END# kgsum f=出生数,死亡数 i=./out_m/step2/df2_base5.csv k=集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名; IN=3492 OUT=3489; 2021/05/18 23:42:42; 2021/05/18 23:42:42


'./out_m/step2/df2_base6.csv'

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,出生数,死亡数
0,2009,01,北海道,01101,札幌市中央区,1580,1571
1,2009,01,北海道,01102,札幌市北区,2052,2057
2,2009,01,北海道,01103,札幌市東区,2221,1860
3,2009,01,北海道,01104,札幌市白石区,1831,1581
4,2009,01,北海道,01105,札幌市豊平区,1721,1540
...,...,...,...,...,...,...,...
3488,2018,47,沖縄県,47382,与那国町,21,20


### STEP3 人口に関するデータ分析向けファイルの作成<a name='STEP3'>

In [24]:
# {コード4.21 住民基本台帳と人口動態調査の前処理データの横結合}
os.makedirs('./out_m/step3', exist_ok=True)

file3_base1='./out_m/step3/df3_base1.csv'
file3_che1='./out_m/step3/df3_che1.csv'
file3_che2='./out_m/step3/df3_che2.csv'

f=None
f<<=nm.mjoin(i=file1_base8, m=file2_base6, N=True, 
                             k='集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名', f='出生数,死亡数',  o=file3_base1)
f.run(msg="on")

display(file3_base1)
display(nm.p(file3_base1, head=10, tail=10))

#差分の調査
f=None
f<<=nm.mdelnull(i=file3_base1, r=True, f='人口_計',  o=file3_che1)
f.run(msg="on")
display(file3_che1)
display(nm.p(file3_che1, head=10, tail=1))

f=None
f<<=nm.mdelnull(i=file3_base1, r=True, f='出生数',  o=file3_che2)
f.run(msg="on")
display(file3_che2)
display(nm.p(file3_che2, head=10, tail=1))

#END# kgjoin -N f=出生数,死亡数 i=./out_m/step1/df1_base8.csv k=集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名 m=./out_m/step2/df2_base6.csv o=./out_m/step3/df3_base1.csv; IN=3486 OUT=3489; 2021/05/18 23:42:42; 2021/05/18 23:42:42


'./out_m/step3/df3_base1.csv'

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数,死亡数
0,2009,01,北海道,01101,札幌市中央区,206252,1580,1571
1,2009,01,北海道,01102,札幌市北区,273577,2052,2057
2,2009,01,北海道,01103,札幌市東区,252688,2221,1860
3,2009,01,北海道,01104,札幌市白石区,203579,1831,1581
4,2009,01,北海道,01105,札幌市豊平区,208476,1721,1540
5,2009,01,北海道,01106,札幌市南区,147397,871,1362
6,2009,01,北海道,01107,札幌市西区,209883,1685,1654
7,2009,01,北海道,01108,札幌市厚別区,129604,819,934
8,2009,01,北海道,01109,札幌市手稲区,138794,953,1091
9,2009,01,北海道,01110,札幌市清田区,114689,773,856


#END# kgdelnull -r f=人口_計 i=./out_m/step3/df3_base1.csv o=./out_m/step3/df3_che1.csv; IN=3489 OUT=3; 2021/05/18 23:42:42; 2021/05/18 23:42:42


'./out_m/step3/df3_che1.csv'

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数,死亡数
0,2010,14,神奈川県,14151,相模原市緑区,,1076,932
1,2010,14,神奈川県,14152,相模原市中央区,,1722,1264
2,2010,14,神奈川県,14153,相模原市南区,,1799,1306


#END# kgdelnull -r f=出生数 i=./out_m/step3/df3_base1.csv o=./out_m/step3/df3_che2.csv; IN=3489 OUT=0; 2021/05/18 23:42:42; 2021/05/18 23:42:42


'./out_m/step3/df3_che2.csv'

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数,死亡数


In [25]:
# {コード4.22 不要な行を削除}
# 相模原市を除去していい理由＝2010年4月に政令指定都市となり、1月・4月締めでデータの集計期間が異なるため
file3_base2='./out_m/step3/df3_base2.csv'

f=None
f<<=nm.mdelnull(i=file3_base1, f='人口_計',  o=file3_base2)
f.run(msg="on")

display(file3_base2)
display(nm.p(file3_base2, head=5, tail=1))

#END# kgdelnull f=人口_計 i=./out_m/step3/df3_base1.csv o=./out_m/step3/df3_base2.csv; IN=3489 OUT=3486; 2021/05/18 23:42:42; 2021/05/18 23:42:42


'./out_m/step3/df3_base2.csv'

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数,死亡数
0,2009,01,北海道,01101,札幌市中央区,206252,1580,1571
1,2009,01,北海道,01102,札幌市北区,273577,2052,2057
2,2009,01,北海道,01103,札幌市東区,252688,2221,1860
3,2009,01,北海道,01104,札幌市白石区,203579,1831,1581
4,2009,01,北海道,01105,札幌市豊平区,208476,1721,1540
...,...,...,...,...,...,...,...,...
3485,2018,47,沖縄県,47382,与那国町,1698,21,20


In [26]:
# {コード4.23 政令指定都市で再集計}
file3_sei1='./out_m/step3/df3_sei1.csv'

f=None
f<<=nm.mjoin(i=file3_base2, m=seirei_mst, k='最新市区町村コード', K='変換前市区町村コード', o=file3_sei1)
f.run(msg="on")

display(file3_sei1)
display(nm.p(file3_sei1, head=5, tail=1))

#END# kgjoin K=変換前市区町村コード i=./out_m/step3/df3_base2.csv k=最新市区町村コード m=./out_m/step0/seireicity_mst.csv o=./out_m/step3/df3_sei1.csv; IN=3486 OUT=374; 2021/05/18 23:42:42; 2021/05/18 23:42:42


'./out_m/step3/df3_sei1.csv'

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数,死亡数,政令指定都市コード,政令指定都市名
0,2009,01,北海道,01101,札幌市中央区,206252,1580,1571,01100,札幌市
1,2011,01,北海道,01101,札幌市中央区,212666,1706,1710,01100,札幌市
2,2012,01,北海道,01101,札幌市中央区,216307,1750,1844,01100,札幌市
3,2013,01,北海道,01101,札幌市中央区,219066,1897,1851,01100,札幌市
4,2014,01,北海道,01101,札幌市中央区,222392,1845,1954,01100,札幌市
...,...,...,...,...,...,...,...,...,...,...
373,2017,14,神奈川県,14153,相模原市南区,269575,2023,2308,14150,相模原市


In [27]:
# {コード4.24 政令指定都市で再集計}
file3_sei2='./out_m/step3/df3_sei2.csv'

f=None
f<<=nm.msum(i=file3_sei1, k='集計年,最新都道府県コード,最新都道府県名,政令指定都市コード,政令指定都市名', f='人口_計,出生数,死亡数')
f<<=nm.mcut(f='集計年,最新都道府県コード,最新都道府県名,政令指定都市コード,政令指定都市名,人口_計,出生数,死亡数', o=file3_sei2)
f.run(msg="on")

display(file3_sei2)
display(nm.p(file3_sei2, head=5, tail=1))

#END# kgcut f=集計年,最新都道府県コード,最新都道府県名,政令指定都市コード,政令指定都市名,人口_計,出生数,死亡数 o=./out_m/step3/df3_sei2.csv; IN=38 OUT=38; 2021/05/18 23:42:42; 2021/05/18 23:42:42
#END# kgsum f=人口_計,出生数,死亡数 i=./out_m/step3/df3_sei1.csv k=集計年,最新都道府県コード,最新都道府県名,政令指定都市コード,政令指定都市名; IN=374 OUT=38; 2021/05/18 23:42:42; 2021/05/18 23:42:42


'./out_m/step3/df3_sei2.csv'

,集計年,最新都道府県コード,最新都道府県名,政令指定都市コード,政令指定都市名,人口_計,出生数,死亡数
0,2009,01,北海道,01100,札幌市,1884939,14506,14506
1,2009,14,神奈川県,14100,横浜市,3605951,32111,25544
2,2009,14,神奈川県,14130,川崎市,1359063,14276,8783
3,2010,01,北海道,01100,札幌市,1891494,14739,15482
4,2010,14,神奈川県,14100,横浜市,3620562,32053,27304
...,...,...,...,...,...,...,...,...
37,2018,14,神奈川県,14150,相模原市,704643,5006,6286


In [28]:
# {コード4.25 結合データとのカラム名統一}
file3_sei3='./out_m/step3/df3_sei3.csv'

f=None
f<<=nm.mfldname(i=file3_sei2, f='政令指定都市コード:最新市区町村コード,政令指定都市名:最新市区町村名', o=file3_sei3)
f.run(msg="on")

display(file3_sei3)
display(nm.p(file3_sei3, head=5, tail=1))

#END# kgfldname f=政令指定都市コード:最新市区町村コード,政令指定都市名:最新市区町村名 i=./out_m/step3/df3_sei2.csv o=./out_m/step3/df3_sei3.csv; IN=38 OUT=38; 2021/05/18 23:42:42; 2021/05/18 23:42:42


'./out_m/step3/df3_sei3.csv'

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数,死亡数
0,2009,01,北海道,01100,札幌市,1884939,14506,14506
1,2009,14,神奈川県,14100,横浜市,3605951,32111,25544
2,2009,14,神奈川県,14130,川崎市,1359063,14276,8783
3,2010,01,北海道,01100,札幌市,1891494,14739,15482
4,2010,14,神奈川県,14100,横浜市,3620562,32053,27304
...,...,...,...,...,...,...,...,...
37,2018,14,神奈川県,14150,相模原市,704643,5006,6286


In [29]:
# {コード4.26 政令指定都市データの縦結合}
file3_base3='./out_m/step3/df3_base3.csv'

f=None
f<<=nm.mcat(i=','.join([file3_base2, file3_sei3]), o=file3_base3)
f.run(msg="on")

display(file3_base3)
display(nm.p(file3_base3, head=5, tail=1))

#END# kgcat i=./out_m/step3/df3_base2.csv,./out_m/step3/df3_sei3.csv o=./out_m/step3/df3_base3.csv; IN=3524 OUT=3524; 2021/05/18 23:42:42; 2021/05/18 23:42:42


'./out_m/step3/df3_base3.csv'

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数,死亡数
0,2009,01,北海道,01101,札幌市中央区,206252,1580,1571
1,2009,01,北海道,01102,札幌市北区,273577,2052,2057
2,2009,01,北海道,01103,札幌市東区,252688,2221,1860
3,2009,01,北海道,01104,札幌市白石区,203579,1831,1581
4,2009,01,北海道,01105,札幌市豊平区,208476,1721,1540
...,...,...,...,...,...,...,...,...
3523,2018,14,神奈川県,14150,相模原市,704643,5006,6286


In [30]:
# {コード4.27 出生率・死亡率を算出}
#スキル：四則演算、round()
file3_base4='./out_m/step3/df3_base4.csv'

f=None
f<<=nm.mcal(i=file3_base3, c='format(round(${出生数}/${人口_計}*100, 0.001), "%.3f")', a='出生率')
f<<=nm.mcal(c='format(round(${死亡数}/${人口_計}*100, 0.001), "%.3f")', a='死亡率', o=file3_base4)
f.run(msg="on")

display(file3_base4)
display(nm.p(file3_base4, head=5, tail=1))

#END# kgcal a=出生率 c=format(round(${出生数}/${人口_計}*100, 0.001), "%.3f") i=./out_m/step3/df3_base3.csv; IN=3524 OUT=3524; 2021/05/18 23:42:43; 2021/05/18 23:42:43
#END# kgcal a=死亡率 c=format(round(${死亡数}/${人口_計}*100, 0.001), "%.3f") o=./out_m/step3/df3_base4.csv; IN=3524 OUT=3524; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/step3/df3_base4.csv'

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数,死亡数,出生率,死亡率
0,2009,01,北海道,01101,札幌市中央区,206252,1580,1571,0.766,0.762
1,2009,01,北海道,01102,札幌市北区,273577,2052,2057,0.750,0.752
2,2009,01,北海道,01103,札幌市東区,252688,2221,1860,0.879,0.736
3,2009,01,北海道,01104,札幌市白石区,203579,1831,1581,0.899,0.777
4,2009,01,北海道,01105,札幌市豊平区,208476,1721,1540,0.826,0.739
...,...,...,...,...,...,...,...,...,...,...
3523,2018,14,神奈川県,14150,相模原市,704643,5006,6286,0.710,0.892


In [31]:
# {コード4.28 列名の修正}
file3_base5='./out_m/step3/df3_base5.csv'

columns={"最新都道府県コード":"都道府県コード",
                     "最新都道府県名":"都道府県名",
                     "最新市区町村コード":"市区町村コード",
                     "最新市区町村名":"市区町村名",
                     "人口_計":"人口数（人）",
                     "出生数":"出生数（人）",
                     "死亡数":"死亡数（人）",
                     "出生率":"出生率（％）",
                     "死亡率":"死亡率（％）"}

f=None
f<<=nm.mfldname(i=file3_base4, f=','.join([k+':'+v  for k,v in columns.items()]), o=file3_base5)
f.run(msg="on")

display(file3_base5)
display(nm.p(file3_base5, head=5, tail=1))

#END# kgfldname f=最新都道府県コード:都道府県コード,最新都道府県名:都道府県名,最新市区町村コード:市区町村コード,最新市区町村名:市区町村名,人口_計:人口数（人）,出生数:出生数（人）,死亡数:死亡数（人）,出生率:出生率（％）,死亡率:死亡率（％） i=./out_m/step3/df3_base4.csv o=./out_m/step3/df3_base5.csv; IN=3524 OUT=3524; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/step3/df3_base5.csv'

,集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,人口数（人）,出生数（人）,死亡数（人）,出生率（％）,死亡率（％）
0,2009,01,北海道,01101,札幌市中央区,206252,1580,1571,0.766,0.762
1,2009,01,北海道,01102,札幌市北区,273577,2052,2057,0.750,0.752
2,2009,01,北海道,01103,札幌市東区,252688,2221,1860,0.879,0.736
3,2009,01,北海道,01104,札幌市白石区,203579,1831,1581,0.899,0.777
4,2009,01,北海道,01105,札幌市豊平区,208476,1721,1540,0.826,0.739
...,...,...,...,...,...,...,...,...,...,...
3523,2018,14,神奈川県,14150,相模原市,704643,5006,6286,0.710,0.892


In [32]:
# {コード4.29 データの並び替え}
# {コード4.30 ファイルの出力}
now = datetime.datetime.now()
file3_out='./out_m/step3/ratio_'+ now.strftime('%Y%m%d') +'_name.csv'

f=None
f<<=nm.msortf(i=file3_base5, f='集計年%n,市区町村コード')
f<<=nm.mfldname(q=True, o=file3_out)
f.run(msg="on")

display(file3_out)
display(nm.p(file3_out, head=10, tail=10))

#END# kgfldname -q o=./out_m/step3/ratio_20210518_name.csv; IN=3524 OUT=3524; 2021/05/18 23:42:43; 2021/05/18 23:42:43
#END# kgsortf f=集計年%n,市区町村コード i=./out_m/step3/df3_base5.csv; IN=3524 OUT=3524; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/step3/ratio_20210518_name.csv'

,集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,人口数（人）,出生数（人）,死亡数（人）,出生率（％）,死亡率（％）
0,2009,01,北海道,01100,札幌市,1884939,14506,14506,0.770,0.770
1,2009,01,北海道,01101,札幌市中央区,206252,1580,1571,0.766,0.762
2,2009,01,北海道,01102,札幌市北区,273577,2052,2057,0.750,0.752
3,2009,01,北海道,01103,札幌市東区,252688,2221,1860,0.879,0.736
4,2009,01,北海道,01104,札幌市白石区,203579,1831,1581,0.899,0.777
5,2009,01,北海道,01105,札幌市豊平区,208476,1721,1540,0.826,0.739
6,2009,01,北海道,01106,札幌市南区,147397,871,1362,0.591,0.924
7,2009,01,北海道,01107,札幌市西区,209883,1685,1654,0.803,0.788
8,2009,01,北海道,01108,札幌市厚別区,129604,819,934,0.632,0.721
9,2009,01,北海道,01109,札幌市手稲区,138794,953,1091,0.687,0.786



<br>

## 章末問題【解答例】<a name='EXE4'>

<br>

*** 
<h3><font color='darkblue'>
1. データフレームdf3_outを用いて，集計年2018年における，出生率TOP20となる市区町村を求めなさい。<a name='EXE4_1'>
</h3>

*** 

[作成条件]
* 項目：集計年、都道府県コード、都道府県名、市区町村コード、市区町村名、出生率（％）、出生率ランキング<br><br>
* ソート順：出生率ランキング<br><br>
* ファイル名：「2018年出生率TOP20[本日日付] [自分の名前].csv」<br><br>

In [33]:
# 2018年のみ
os.makedirs('./out_m/exercize', exist_ok=True)
file_ex4_1_1='./out_m/exercize/df_ex4_1_1.csv'

f=None
f<<=nm.mselstr(i=file3_out, f='集計年', v='2018', o=file_ex4_1_1)
f.run(msg="on")

display(file_ex4_1_1)
display(nm.p(file_ex4_1_1, head=5, tail=1))

#END# kgselstr f=集計年 i=./out_m/step3/ratio_20210518_name.csv o=./out_m/exercize/df_ex4_1_1.csv v=2018; IN=3524 OUT=353; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/exercize/df_ex4_1_1.csv'

,集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,人口数（人）,出生数（人）,死亡数（人）,出生率（％）,死亡率（％）
0,2018,01,北海道,01100,札幌市,1940035,13248,19343,0.683,0.997
1,2018,01,北海道,01101,札幌市中央区,231394,1744,2174,0.754,0.940
2,2018,01,北海道,01102,札幌市北区,282525,1847,2842,0.654,1.006
3,2018,01,北海道,01103,札幌市東区,260507,1978,2512,0.759,0.964
4,2018,01,北海道,01104,札幌市白石区,210840,1711,2032,0.812,0.964
...,...,...,...,...,...,...,...,...,...,...
352,2018,47,沖縄県,47382,与那国町,1698,21,20,1.237,1.178


In [34]:
#並び替え
file_ex4_1_2='./out_m/exercize/df_ex4_1_2.csv'

f=None
f<<=nm.msortf(i=file_ex4_1_1, f='出生率（％）%nr', o=file_ex4_1_2)
f.run(msg="on")

display(file_ex4_1_2)
display(nm.p(file_ex4_1_2, head=5, tail=1))

#END# kgsortf f=出生率（％）%nr i=./out_m/exercize/df_ex4_1_1.csv o=./out_m/exercize/df_ex4_1_2.csv; IN=353 OUT=353; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/exercize/df_ex4_1_2.csv'

,集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,人口数（人）,出生数（人）,死亡数（人）,出生率（％）,死亡率（％）
0,2018,47,沖縄県,47358,北大東村,568,12,4,2.113,0.704
1,2018,47,沖縄県,47350,南風原町,38441,564,229,1.467,0.596
2,2018,47,沖縄県,47353,渡嘉敷村,691,10,8,1.447,1.158
3,2018,47,沖縄県,47348,与那原町,19477,281,133,1.443,0.683
4,2018,13,東京都,13102,中央区,149832,2043,911,1.364,0.608
...,...,...,...,...,...,...,...,...,...,...
352,2018,13,東京都,13402,青ヶ島村,166,0,0,0.000,0.000


In [35]:
#ランキング付与
file_ex4_1_3='./out_m/exercize/df_ex4_1_3.csv'

f=None
# ※nm.mnumberではパラメータs=で並び替えを行ってからランキングを付与するため, 本来は直前セルで行っている並び替えは不要
f<<=nm.mnumber(i=file_ex4_1_2, a='出生率ランキング', s='出生率（％）%nr', S=1, e='skip')
f<<=nm.msortf(f='出生率ランキング%n', o=file_ex4_1_3)
f.run(msg="on")

display(file_ex4_1_3)
display(nm.p(file_ex4_1_3, head=5, tail=1))

#END# kgnumber S=1 a=出生率ランキング e=skip i=./out_m/exercize/df_ex4_1_2.csv s=出生率（％）%nr; IN=353 OUT=353; 2021/05/18 23:42:43; 2021/05/18 23:42:43
#END# kgsortf f=出生率ランキング%n o=./out_m/exercize/df_ex4_1_3.csv; IN=353 OUT=353; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/exercize/df_ex4_1_3.csv'

,集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,人口数（人）,出生数（人）,死亡数（人）,出生率（％）,死亡率（％）,出生率ランキング
0,2018,47,沖縄県,47358,北大東村,568,12,4,2.113,0.704,1
1,2018,47,沖縄県,47350,南風原町,38441,564,229,1.467,0.596,2
2,2018,47,沖縄県,47353,渡嘉敷村,691,10,8,1.447,1.158,3
3,2018,47,沖縄県,47348,与那原町,19477,281,133,1.443,0.683,4
4,2018,13,東京都,13102,中央区,149832,2043,911,1.364,0.608,5
...,...,...,...,...,...,...,...,...,...,...,...
352,2018,13,東京都,13402,青ヶ島村,166,0,0,0.000,0.000,353


In [36]:
# 20位以下を抽出
file_ex4_1_4='./out_m/exercize/df_ex4_1_4.csv'

f=None
f<<=nm.msel(i=file_ex4_1_3, c='${出生率ランキング}<=20', o=file_ex4_1_4)
f.run(msg="on")

display(file_ex4_1_4)
display(nm.p(file_ex4_1_4, head=5, tail=1))

#END# kgsel c=${出生率ランキング}<=20 i=./out_m/exercize/df_ex4_1_3.csv o=./out_m/exercize/df_ex4_1_4.csv; IN=353 OUT=20; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/exercize/df_ex4_1_4.csv'

,集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,人口数（人）,出生数（人）,死亡数（人）,出生率（％）,死亡率（％）,出生率ランキング
0,2018,47,沖縄県,47358,北大東村,568,12,4,2.113,0.704,1
1,2018,47,沖縄県,47350,南風原町,38441,564,229,1.467,0.596,2
2,2018,47,沖縄県,47353,渡嘉敷村,691,10,8,1.447,1.158,3
3,2018,47,沖縄県,47348,与那原町,19477,281,133,1.443,0.683,4
4,2018,13,東京都,13102,中央区,149832,2043,911,1.364,0.608,5
...,...,...,...,...,...,...,...,...,...,...,...
19,2018,14,神奈川県,14133,川崎市中原区,245239,2816,1587,1.148,0.647,20


In [37]:
# カラム選択, CSV出力
now = datetime.datetime.now()
file_ex4_1a='./out_m/exercize/2018年出生率TOP20_'+ now.strftime('%Y%m%d') +'_a.csv'

f=None
f<<=nm.mcut(i=file_ex4_1_4, f='集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,出生率（％）,出生率ランキング', o=file_ex4_1a)
f.run(msg="on")

display(file_ex4_1a)
display(nm.p(file_ex4_1a, head=5, tail=1))

#END# kgcut f=集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,出生率（％）,出生率ランキング i=./out_m/exercize/df_ex4_1_4.csv o=./out_m/exercize/2018年出生率TOP20_20210518_a.csv; IN=20 OUT=20; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/exercize/2018年出生率TOP20_20210518_a.csv'

,集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,出生率（％）,出生率ランキング
0,2018,47,沖縄県,47358,北大東村,2.113,1
1,2018,47,沖縄県,47350,南風原町,1.467,2
2,2018,47,沖縄県,47353,渡嘉敷村,1.447,3
3,2018,47,沖縄県,47348,与那原町,1.443,4
4,2018,13,東京都,13102,中央区,1.364,5
...,...,...,...,...,...,...,...
19,2018,14,神奈川県,14133,川崎市中原区,1.148,20


In [38]:
# nysol_pythonでは上記処理をパイプ処理でつながて一気に行う事ができる
now = datetime.datetime.now()
file_ex4_1b='./out_m/exercize/2018年出生率TOP20_'+ now.strftime('%Y%m%d') +'_b.csv'
f=None
f<<=nm.mselstr(i=file3_out, f='集計年', v='2018')
f<<=nm.mnumber(a='出生率ランキング', s='出生率（％）%nr', S=1, e='skip')
f<<=nm.msel(c='${出生率ランキング}<=20')
f<<=nm.mcut(f='集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,出生率（％）,出生率ランキング')
f<<=nm.msortf(f='出生率ランキング%n', o=file_ex4_1b)
f.run(msg="on")

display(file_ex4_1b)
display(nm.p(file_ex4_1b, head=5, tail=1))

#END# kgsel c=${出生率ランキング}<=20; IN=353 OUT=20; 2021/05/18 23:42:43; 2021/05/18 23:42:43
#END# kgnumber S=1 a=出生率ランキング e=skip s=出生率（％）%nr; IN=353 OUT=353; 2021/05/18 23:42:43; 2021/05/18 23:42:43
#END# kgselstr f=集計年 i=./out_m/step3/ratio_20210518_name.csv v=2018; IN=3524 OUT=353; 2021/05/18 23:42:43; 2021/05/18 23:42:43
#END# kgcut f=集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,出生率（％）,出生率ランキング; IN=20 OUT=20; 2021/05/18 23:42:43; 2021/05/18 23:42:43
#END# kgsortf f=出生率ランキング%n o=./out_m/exercize/2018年出生率TOP20_20210518_b.csv; IN=20 OUT=20; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/exercize/2018年出生率TOP20_20210518_b.csv'

,集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,出生率（％）,出生率ランキング
0,2018,47,沖縄県,47358,北大東村,2.113,1
1,2018,47,沖縄県,47350,南風原町,1.467,2
2,2018,47,沖縄県,47353,渡嘉敷村,1.447,3
3,2018,47,沖縄県,47348,与那原町,1.443,4
4,2018,13,東京都,13102,中央区,1.364,5
...,...,...,...,...,...,...,...
19,2018,14,神奈川県,14133,川崎市中原区,1.148,20


[TOPに戻る](#S4_e)

<br>

*** 
<h3><font color='darkblue'>
2. データフレームdf3_base2 を用いて，都道府県集計のデータフレームを作成しなさい。<a name='EXE4_2'>
</h3>

*** 

In [39]:
file_ex4_2='./out_m/exercize/df_ex4_2.csv'

f=None
f<<=nm.msum(i=file3_base2, k='集計年,最新都道府県コード', f='人口_計,出生数,死亡数')
f<<=nm.mcut(f='集計年,最新都道府県コード,人口_計,出生数,死亡数', o=file_ex4_2)
f.run(msg="on")

display(file_ex4_2)
display(nm.p(file_ex4_2, head=5, tail=1))

#END# kgsum f=人口_計,出生数,死亡数 i=./out_m/step3/df3_base2.csv k=集計年,最新都道府県コード; IN=3486 OUT=40; 2021/05/18 23:42:43; 2021/05/18 23:42:43
#END# kgcut f=集計年,最新都道府県コード,人口_計,出生数,死亡数 o=./out_m/exercize/df_ex4_2.csv; IN=40 OUT=40; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/exercize/df_ex4_2.csv'

,集計年,最新都道府県コード,人口_計,出生数,死亡数
0,2009,01,5543556,40165,53221
1,2009,13,12548258,106613,98304
2,2009,14,8848329,78057,63745
3,2009,47,1397812,16744,9923
4,2010,01,5520894,40158,55404
...,...,...,...,...,...
39,2018,47,1456122,15732,12157


[TOPに戻る](#S4_e)

<br>

*** 
<h3><font color='darkblue'>
3. データフレームdf2_base3から，都道府県集計のレコードを抽出しなさい。<a name='EXE4_3'>
</h3>

*** 

In [40]:
# 地域情報に全角数字を含んでいるのが都道府県レコード
file_ex4_3='./out_m/exercize/df_ex4_3.csv'

f=None
f<<=nm.msel(i=file2_base3, c='regexsw($s{地域情報}, "[０-９]+")')
f<<=nm.msortf(f='地域情報,集計年', o=file_ex4_3)
f.run(msg="on")

display(file_ex4_3)
display(nm.p(file_ex4_3, head=5, tail=1))

#END# kgsel c=regexsw($s{地域情報}, "[０-９]+") i=./out_m/step2/df2_base3.csv; IN=4507 OUT=40; 2021/05/18 23:42:43; 2021/05/18 23:42:43
#END# kgsortf f=地域情報,集計年 o=./out_m/exercize/df_ex4_3.csv; IN=40 OUT=40; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/exercize/df_ex4_3.csv'

,地域情報,出生数,出生数_2500g未満,死亡数,死亡数_乳児,死亡数_新生児,死産数_自然死産,死産数_人工死産,周産期死亡_総数,周産期死亡_22週以後の死産数,周産期死亡_早期新生児死亡数,婚姻件数,離婚件数,集計年,市区町村コード
0,０１北海道,40165,3670,53221,89,40,569,792,178,147,31,28271,12294,2009,０１北海道
1,０１北海道,40158,3934,55404,84,41,512,748,171,138,33,28389,12596,2010,０１北海道
2,０１北海道,39292,3804,56970,84,41,513,796,168,138,30,26518,11847,2011,０１北海道
3,０１北海道,38686,3764,58066,88,43,473,704,160,130,30,26538,11593,2012,０１北海道
4,０１北海道,38190,3722,59432,85,43,398,736,138,104,34,26330,11285,2013,０１北海道
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,４７沖縄,15732,1728,12157,24,11,182,200,53,46,7,7887,3618,2018,４７沖縄


[TOPに戻る](#S4_e)

<br>

*** 
<h3><font color='darkblue'>
4. 問題(2) と問題(3) の結果を比較し，差異が2010年神奈川県に存在することを確認しなさい。<a name='EXE4_4'>
</h3>

*** 

In [41]:
# 比較用にカラム名の変更
file_ex4_2_1='./out_m/exercize/df_ex4_2_1.csv'

f=None
f<<=nm.mfldname(i=file_ex4_2, f='最新都道府県コード:都道府県コード,出生数:出生数1', o=file_ex4_2_1)
f.run(msg="on")

display(file_ex4_2_1)
display(nm.p(file_ex4_2_1, head=5, tail=1))

#END# kgfldname f=最新都道府県コード:都道府県コード,出生数:出生数1 i=./out_m/exercize/df_ex4_2.csv o=./out_m/exercize/df_ex4_2_1.csv; IN=40 OUT=40; 2021/05/18 23:42:43; 2021/05/18 23:42:43


'./out_m/exercize/df_ex4_2_1.csv'

,集計年,都道府県コード,人口_計,出生数1,死亡数
0,2009,01,5543556,40165,53221
1,2009,13,12548258,106613,98304
2,2009,14,8848329,78057,63745
3,2009,47,1397812,16744,9923
4,2010,01,5520894,40158,55404
...,...,...,...,...,...
39,2018,47,1456122,15732,12157


In [42]:
# 比較用の都道府県コードの付与
file_ex4_3_1='./out_m/exercize/df_ex4_3_1.csv'

f=None
f<<=nm.mcal(i=file_ex4_3, c='leftw($s{地域情報}, 2)', a='都道府県コード')
f<<=nm.msed(f='都道府県コード', c='０', v='0', g=True)
f<<=nm.msed(f='都道府県コード', c='１', v='1', g=True)
f<<=nm.msed(f='都道府県コード', c='２', v='2', g=True)
f<<=nm.msed(f='都道府県コード', c='３', v='3', g=True)
f<<=nm.msed(f='都道府県コード', c='４', v='4', g=True)
f<<=nm.msed(f='都道府県コード', c='５', v='5', g=True)
f<<=nm.msed(f='都道府県コード', c='６', v='6', g=True)
f<<=nm.msed(f='都道府県コード', c='７', v='7', g=True)
f<<=nm.msed(f='都道府県コード', c='８', v='8', g=True)
f<<=nm.msed(f='都道府県コード', c='９', v='9', g=True, o=file_ex4_3_1)
f.run(msg="on")


display(file_ex4_3_1)
display(nm.p(file_ex4_3_1, head=5, tail=1))

#END# kgsed -g c=６ f=都道府県コード v=6; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44
#END# kgsed -g c=５ f=都道府県コード v=5; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44
#END# kgsed -g c=４ f=都道府県コード v=4; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44
#END# kgsed -g c=３ f=都道府県コード v=3; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44
#END# kgsed -g c=２ f=都道府県コード v=2; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44
#END# kgsed -g c=１ f=都道府県コード v=1; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44
#END# kgsed -g c=０ f=都道府県コード v=0; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44
#END# kgcal a=都道府県コード c=leftw($s{地域情報}, 2) i=./out_m/exercize/df_ex4_3.csv; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44
#END# kgsed -g c=９ f=都道府県コード o=./out_m/exercize/df_ex4_3_1.csv v=9; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44
#END# kgsed -g c=８ f=都道府県コード v=8; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44
#END# kgsed -g c=７ f=都道府県コード v=7; 

'./out_m/exercize/df_ex4_3_1.csv'

,地域情報,出生数,出生数_2500g未満,死亡数,死亡数_乳児,死亡数_新生児,死産数_自然死産,死産数_人工死産,周産期死亡_総数,周産期死亡_22週以後の死産数,周産期死亡_早期新生児死亡数,婚姻件数,離婚件数,集計年,市区町村コード,都道府県コード
0,０１北海道,40165,3670,53221,89,40,569,792,178,147,31,28271,12294,2009,０１北海道,01
1,０１北海道,40158,3934,55404,84,41,512,748,171,138,33,28389,12596,2010,０１北海道,01
2,０１北海道,39292,3804,56970,84,41,513,796,168,138,30,26518,11847,2011,０１北海道,01
3,０１北海道,38686,3764,58066,88,43,473,704,160,130,30,26538,11593,2012,０１北海道,01
4,０１北海道,38190,3722,59432,85,43,398,736,138,104,34,26330,11285,2013,０１北海道,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,４７沖縄,15732,1728,12157,24,11,182,200,53,46,7,7887,3618,2018,４７沖縄,47


In [43]:
# 比較用カラムのみ保持
file_ex4_2_2='./out_m/exercize/df_ex4_2_2.csv'
f=None
f<<=nm.mcut(i=file_ex4_2_1, f='集計年,都道府県コード,出生数1', o=file_ex4_2_2)
f.run(msg="on")
display(file_ex4_2_2)
display(nm.p(file_ex4_2_2, head=5, tail=1))

file_ex4_3_2='./out_m/exercize/df_ex4_3_2.csv'
f=None
f<<=nm.mcut(i=file_ex4_3_1, f='集計年,都道府県コード,出生数', o=file_ex4_3_2)
f.run(msg="on")
display(file_ex4_3_2)
display(nm.p(file_ex4_3_2, head=5, tail=1))


# 結合
file_ex4_4_1='./out_m/exercize/df_ex4_4_1.csv'
f=None
f<<=nm.mnjoin(i=file_ex4_2_2, m=file_ex4_3_2, k='集計年,都道府県コード', n=True, N=True, o=file_ex4_4_1)
f.run(msg="on")
display(file_ex4_4_1)
display(nm.p(file_ex4_4_1, head=5, tail=1))

# 出生数が異
file_ex4_4='./out_m/exercize/df_ex4_4.csv'
f=None
f<<=nm.msel(i=file_ex4_4_1, c='${出生数}!=${出生数1}', o=file_ex4_4)
f.run(msg="on")
display(file_ex4_4)
display(nm.p(file_ex4_4, head=5, tail=1))

#END# kgcut f=集計年,都道府県コード,出生数1 i=./out_m/exercize/df_ex4_2_1.csv o=./out_m/exercize/df_ex4_2_2.csv; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44


'./out_m/exercize/df_ex4_2_2.csv'

,集計年,都道府県コード,出生数1
0,2009,01,40165
1,2009,13,106613
2,2009,14,78057
3,2009,47,16744
4,2010,01,40158
...,...,...,...
39,2018,47,15732


#END# kgcut f=集計年,都道府県コード,出生数 i=./out_m/exercize/df_ex4_3_1.csv o=./out_m/exercize/df_ex4_3_2.csv; IN=40 OUT=40; 2021/05/18 23:42:44; 2021/05/18 23:42:44


'./out_m/exercize/df_ex4_3_2.csv'

...

'./out_m/exercize/df_ex4_4_1.csv'

,集計年,都道府県コード,出生数1,出生数
0,2009,01,40165,40165
1,2009,13,106613,106613
2,2009,14,78057,78057
3,2009,47,16744,16744
4,2010,01,40158,40158
...,...,...,...,...
39,2018,47,15732,15732


#END# kgsel c=${出生数}!=${出生数1} i=./out_m/exercize/df_ex4_4_1.csv o=./out_m/exercize/df_ex4_4.csv; IN=40 OUT=1; 2021/05/18 23:42:44; 2021/05/18 23:42:44


'./out_m/exercize/df_ex4_4.csv'

,集計年,都道府県コード,出生数1,出生数
0,2010,14,73480,78077


[TOPに戻る](#S4_e)
<br>

*** 
<h3><font color='darkblue'>
5. 問題(4) にて，差分が2010年神奈川県にのみ存在した理由を述べなさい。<a name='EXE4_5'>
</h3>

*** 

##### 解答例)

本文中でも触れているが、相模原市は2010年4月1日に政令指定都市となった。
（2）で作成したものは政令指定都市のレコードしか存在しておらず、そのデータを都道府県集計すると、2010年1月～3月の相模原市の数が計上されていない．
そのため、（3）で作成した1月～3月の数字も計上されている都道府県集計と差分が発生した．

<br>

[TOPに戻る](#S4_e)
